## response collection

The purpose of this notebook is to compare the model predictions saved to a mongodb
collection with other sources of information about the site in the model at the time
of the astronomical twilight in the response.


In [50]:
import typing as t
from dataclasses import dataclass
from pathlib import Path
from datetime import datetime

import httpx
from pymongo import MongoClient

DB_NAME = "validation_data"
COLLECTION_NAME = "api_response"

coordinates = (-30.2466, -70.7494)
lat, lon = coordinates


@dataclass
class Site:
    lat: str
    lon: str
    astro_twilight: str
    astro_twilight_hour: int


with open(Path.cwd() / ".env") as f:
    first_line, *_ = f.read().split("\n")
    idx = first_line.find("=")
    uri = first_line[idx + 1 :]


def get_db():
    client = MongoClient(uri)
    return client[DB_NAME]


db = get_db()
db

Database(MongoClient(host=['ac-atnkhx8-shard-00-00.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-02.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-01.7a9iiun.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, ssl_cert_reqs=<VerifyMode.CERT_NONE: 0>, authsource='admin', replicaset='atlas-3ats48-shard-0'), 'validation_data')

In [51]:
collection = db[COLLECTION_NAME]
collection

Collection(Database(MongoClient(host=['ac-atnkhx8-shard-00-00.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-02.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-01.7a9iiun.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, ssl_cert_reqs=<VerifyMode.CERT_NONE: 0>, authsource='admin', replicaset='atlas-3ats48-shard-0'), 'validation_data'), 'api_response')

In [52]:
async def get_prediction(coordinates: t.Tuple):
    async with httpx.AsyncClient() as client:
        url = f"http://localhost:8000/api/prediction?lat={lat}&lon={lon}"
        r = await client.get(url)
        return r.json()


async def insert_prediction():
    prediction = await get_prediction(coordinates)
    collection.insert_one(prediction)
    return prediction


prediction = await insert_prediction()
prediction

{'brightness_mpsas': 19.7033,
 'astro_twilight': {'iso': '2023-12-31 01:24:12.448', 'type': 'nearest'},
 '_id': ObjectId('6590c6deb29b8604829db6c8')}

In [53]:
items_in_collection = list(collection.find())
brightness_values = [item["brightness_mpsas"] for item in items_in_collection]
brightness_values

[19.7034, 19.7034, 19.7033, 19.7034, 19.7034, 19.7034, 19.7034, 19.7033]

In [59]:
open_meteo_url = "https://archive-api.open-meteo.com/v1"


async def get_historical_meteo_data_at_site(site: Site) -> t.Any:
    url = f"{open_meteo_url}/archive?latitude={site.lat}&longitude={site.lon}&start_date=2023-12-15&end_date=2023-12-16&hourly=temperature_2m,cloud_cover"
    async with httpx.AsyncClient() as client:
        r = await client.get(url)
        r.raise_for_status()
        json = r.json()
        return json["hourly"]["cloud_cover"][site.astro_twilight_hour]


astro_twilight_iso = prediction["astro_twilight"]["iso"]
s = Site(
    lat=lat,
    lon=lon,
    astro_twilight=astro_twilight_iso,
    astro_twilight_hour=datetime.strptime(
        astro_twilight_iso, "%Y-%m-%d %H:%M:%S.%f"
    ).hour,
)
await get_historical_meteo_data_at_site(s)

0